# Train and deploy a model for the Amazon Bin Image counting problem

This notebook contains code to perform hyperparameter tuning, training, debugging of a multi-class image classifier for for counting objects in images from bins in Amazon distribution centers. During the EDA phase the problem has been reduced to a 


The model is implemented using PyTorch. The training data comes directly from the public dataset at https://github.com/awslabs/open-data-docs/tree/main/docs/aft-vbi-pds. The original dataset contains >500,000 images, but in this project a randomly selected subsample of 50,000 images is used. See EDA for details. 

## Install debugging tools

Run only once.

In [1]:
!pip install smdebug 

/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
  Using cached smdebug-1.0.12-py2.py3-none-any.whl (270 kB)
  Using cached pyinstrument-3.4.2-py2.py3-none-any.whl (83 kB)
  Using cached pyinstrument_cext-0.2.4-cp37-cp37m-manylinux2010_x86_64.whl (20 kB)


## Create S3 bucket for the models

Run only once.

In [ ]:
import boto3 

In [ ]:
client = boto3.client('s3')
response = client.create_bucket(
    ACL='private',
    Bucket='amazon-bin-images-models',
)

## Import useful modules

In [2]:
import os
import sys
import io
import json
import IPython
import numpy as np
import pandas as pd
from PIL import Image
import boto3
import sagemaker
from sagemaker.pytorch import PyTorch
from sagemaker.tuner import (
    IntegerParameter,
    CategoricalParameter,
    ContinuousParameter,
    HyperparameterTuner,
)
from sagemaker.debugger import Rule, ProfilerRule, rule_configs
from sagemaker.debugger import DebuggerHookConfig, CollectionConfig, ProfilerConfig, FrameworkProfile
from smdebug.trials import create_trial
from smdebug.core.modes import ModeKeys
from smdebug.profiler.analysis.notebook_utils.training_job import TrainingJob
from smdebug.profiler.analysis.notebook_utils.timeline_charts import TimelineCharts

[2022-01-19 01:30:51.479 datascience-1-0-ml-t3-medium-1abf3407f667f989be9d86559395:61 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None


## Hyperparameter Tuning

The code below sets up a series of hyperparameter sweeps to find the set that achieves the best model performance. The latter is quantified by a series of metrics computed from each dataset: 
- Average training loss.
- Training accuracy.
- Training RMSE.
- Average validation loss.
- Validation accuracy.
- Validation RMSE.
- Average testing loss.
- Testing accuracy.
- Testing RMSE.

**The average validation loss is the metric to minimize to perform hyperparameter tuning.**

In [3]:
# Hyperparameter ranges
hyperparameter_ranges = {
    "learning_rate": ContinuousParameter(0.001, 0.1),
    "batch_size": CategoricalParameter([32, 64, 128]),
    "epochs": IntegerParameter(2, 10)
}

# Objective metric 
objective_metric_name = "Validation Loss"
objective_type = "Minimize"
metric_definitions = [
    {"Name": "Training Loss", "Regex": "Training Loss: ([0-9\\.]+)"},
    {"Name": "Training Accuracy", "Regex": "Training Accuracy: ([0-9\\.]+)"},
    {"Name": "Training RMSE", "Regex": "Training RMSE: ([0-9\\.]+)"},
    {"Name": "Validation Loss", "Regex": "Validation Loss: ([0-9\\.]+)"},
    {"Name": "Validation Accuracy", "Regex": "Validation Accuracy: ([0-9\\.]+)"},
    {"Name": "Validation RMSE", "Regex": "Validation RMSE: ([0-9\\.]+)"},
    {"Name": "Testing Loss", "Regex": "Testing Loss: ([0-9\\.]+)"},
    {"Name": "Testing Accuracy", "Regex": "Testing Accuracy: ([0-9\\.]+)"},
    {"Name": "Testing RMSE", "Regex": "Testing RMSE: ([0-9\\.]+)"},
]

In [3]:
estimator = PyTorch(
    role=sagemaker.get_execution_role(),
    base_job_name='amazon-bin-images-pytorch',
    instance_count=1,
    instance_type="ml.g4dn.xlarge", 
    source_dir="./",
    entry_point="hyperparameter_opt.py",
    framework_version="1.8",
    py_version="py36",
)
tuner = HyperparameterTuner(
    estimator,
    objective_metric_name,
    hyperparameter_ranges,
    metric_definitions,
    max_jobs=6,
    max_parallel_jobs=2,
    objective_type=objective_type
)

In [ ]:
os.environ['SM_CHANNEL_TRAINING']='s3://amazon-bin-images-sub'
os.environ['SM_MODEL_DIR']='s3://amazon-bin-images-models'
os.environ['SM_OUTPUT_DATA_DIR']='s3://amazon-bin-images-models'
tuner.fit({"training": "s3://amazon-bin-images-sub"})

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

## Describe tuning results

In [8]:
from sagemaker.analytics import HyperparameterTuningJobAnalytics

exp = HyperparameterTuningJobAnalytics(
  hyperparameter_tuning_job_name='pytorch-training-220118-0735'
)

jobs = exp.dataframe()

jobs.sort_values('FinalObjectiveValue', ascending=0)

,batch_size,epochs,learning_rate,TrainingJobName,TrainingJobStatus,FinalObjectiveValue,TrainingStartTime,TrainingEndTime,TrainingElapsedTimeSeconds
3,"""128""",10.0,0.033175,pytorch-training-220118-0735-003-95e0e787,Completed,2.175099,2022-01-18 08:28:30+00:00,2022-01-18 09:40:42+00:00,4332.0
5,"""32""",7.0,0.018900,pytorch-training-220118-0735-001-508d3b5e,Completed,2.170819,2022-01-18 07:38:08+00:00,2022-01-18 08:29:04+00:00,3056.0
0,"""64""",5.0,0.003215,pytorch-training-220118-0735-006-02ebce65,Completed,2.017632,2022-01-18 09:43:32+00:00,2022-01-18 10:21:59+00:00,2307.0
4,"""32""",6.0,0.005200,pytorch-training-220118-0735-002-c4b2ddda,Completed,2.009075,2022-01-18 07:38:20+00:00,2022-01-18 08:23:22+00:00,2702.0
2,"""128""",8.0,0.003670,pytorch-training-220118-0735-004-22014cd8,Completed,1.970061,2022-01-18 08:31:44+00:00,2022-01-18 09:30:06+00:00,3502.0
1,"""128""",5.0,0.001495,pytorch-training-220118-0735-005-021f6137,Completed,1.964834,2022-01-18 09:42:01+00:00,2022-01-18 10:22:21+00:00,2420.0


In [9]:
jobs.to_csv('hyperparameter_tuning_results.csv',index=False)

## Select best model hyperparameters

In [4]:
# From tuner object
#best_estimator=tuner.best_estimator()

# If kernel dies - retrieve from completed training job
# From log file
jobs = pd.read_csv('hyperparameter_tuning_results.csv')
ibest = np.argmin(jobs.FinalObjectiveValue)
best_training_job_name = jobs.loc[ibest,'TrainingJobName']
best_estimator = sagemaker.estimator.Estimator.attach(best_training_job_name)


2022-01-18 10:22:21 Starting - Preparing the instances for training
2022-01-18 10:22:21 Downloading - Downloading input data
2022-01-18 10:22:21 Training - Training image download completed. Training in progress.
2022-01-18 10:22:21 Uploading - Uploading generated training model
2022-01-18 10:22:21 Completed - Training job completed


In [5]:
best_estimator.hyperparameters()

{'_tuning_objective_metric': 'Validation Loss',
 'batch_size': '"128"',
 'epochs': '5',
 'learning_rate': '0.0014953389429079192',
 'sagemaker_container_log_level': '20',
 'sagemaker_estimator_class_name': '"PyTorch"',
 'sagemaker_estimator_module': '"sagemaker.pytorch.estimator"',
 'sagemaker_job_name': '"amazon-bin-images-pytorch-2022-01-18-07-35-28-266"',
 'sagemaker_program': '"hyperparameter_opt.py"',
 'sagemaker_region': '"us-east-1"',
 'sagemaker_submit_directory': '"s3://sagemaker-us-east-1-418872927507/amazon-bin-images-pytorch-2022-01-18-07-35-28-266/source/sourcedir.tar.gz"'}

In [6]:
hyperparameters = {
    "batch_size": int(best_estimator.hyperparameters()['batch_size'].replace('"', '')), 
    "learning_rate": best_estimator.hyperparameters()['learning_rate'], 
    "epochs": best_estimator.hyperparameters()['epochs']
}

## Profiler and Debugger settings

In [7]:
rules = [
    Rule.sagemaker(rule_configs.vanishing_gradient()),
    Rule.sagemaker(rule_configs.overfit()),
    Rule.sagemaker(rule_configs.overtraining()),
    Rule.sagemaker(rule_configs.poor_weight_initialization()),
    ProfilerRule.sagemaker(rule_configs.ProfilerReport()),
]

In [8]:
hook_config = DebuggerHookConfig(
    hook_parameters={
        "train.save_interval": "1",
        "eval.save_interval": "1"
    }
)

profiler_config = ProfilerConfig(
    system_monitor_interval_millis=500, framework_profile_params=FrameworkProfile(num_steps=1)
)

## Train model with best hyperparameters, debugger and profiler

In [13]:
mi_estimator = PyTorch(
    role=sagemaker.get_execution_role(),
    base_job_name='amazon-bin-images-pytorch',
    instance_count=1,
    instance_type="ml.g4dn.xlarge", 
    source_dir="./",
    entry_point="train.py",
    framework_version="1.8",
    py_version="py36",
    hyperparameters=hyperparameters,
    ## Debugger and Profiler parameters
    rules = rules,
    debugger_hook_config=hook_config,
    profiler_config=profiler_config
)

In [14]:
os.environ['SM_CHANNEL_TRAINING']='s3://amazon-bin-images-sub'
os.environ['SM_MODEL_DIR']='s3://amazon-bin-images-models'
os.environ['SM_OUTPUT_DATA_DIR']='s3://amazon-bin-images-models'
mi_estimator.fit({"training": "s3://amazon-bin-images-sub"})

2022-01-19 02:17:25 Starting - Starting the training job...
2022-01-19 02:17:54 Starting - Launching requested ML instancesVanishingGradient: InProgress
Overfit: InProgress
Overtraining: InProgress
PoorWeightInitialization: InProgress
ProfilerReport: InProgress
......
2022-01-19 02:18:54 Starting - Preparing the instances for training.........
2022-01-19 02:20:15 Downloading - Downloading input data.............................................
2022-01-19 02:27:58 Training - Training image download completed. Training in progress.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2022-01-19 02:27:57,823 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2022-01-19 02:27:57,843 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2022-01-19 02:27:57,852 sagemaker_pytorch_container.training INFO     Invoking user training script.
2022-01-19 02:27:58,601 

## Locate model artifacts

In [15]:
model_location=mi_estimator.model_data

In [16]:
model_location

's3://sagemaker-us-east-1-418872927507/amazon-bin-images-pytorch-2022-01-19-02-17-24-354/output/model.tar.gz'

## Prepare Deployment

In [ ]:
jpeg_serializer = sagemaker.serializers.IdentitySerializer("image/jpeg")
json_deserializer = sagemaker.deserializers.JSONDeserializer()

class ImagePredictor(Predictor):
    def __init__(self, endpoint_name, sagemaker_session):
        super(ImagePredictor, self).__init__(
            endpoint_name,
            sagemaker_session=sagemaker_session,
            serializer=jpeg_serializer,
            deserializer=json_deserializer,
        )

In [ ]:
mi_pytorch_model = PyTorchModel(
    model_data=model_location, 
    role=role, 
    entry_point='inference.py',
    py_version='py36',
    framework_version='1.8',
    predictor_cls=ImagePredictor
)

In [ ]:
predictor = mi_pytorch_model.deploy(initial_instance_count=1, instance_type='ml.m5.large')

## Test deployed model 

In [ ]:
img_bytes = 'ENTER SOMETHING HERE'
type(img_bytes)

In [ ]:
Image.open(io.BytesIO(img_bytes))

In [ ]:
response=predictor.predict(img_bytes, initial_args={"ContentType": "image/jpeg"})

In [ ]:
np.argmax(response, 1)